# ASVspoof 2017 v2.0 evaluation dataset meta-data analysis

> Color code specifies the level of difficulty
 1. **Green** indicates the easiest case to be detected by the system
 1. **Yellow** - medium level
 1. **Red** - difficult level
 
 
Using the following end-to-end CNN model (the one we submitted in INTERSPEECH), we compute EER for different qualities of environment, playback and the recording devices

> models_After_ICASSP/InterSpeech2018_v2.0/testing_best_model_with3sec_RELU/keep_0.5_0.5_relurun8 

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline

## Environment ID List :

1. Red color - the hardest ones (Total = 5)

['E01','E23','E24','E25','E26']

2. Green color - the easiest to detect as it is the noisy ones: total = 3

['E02','E03','E06']

3. Yello color - the medium : total = 18

['E04','E05','E07','E08','E09','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19','E20','E21','E22']
    

In [27]:
yellowE = ['E04','E05','E07','E08','E09','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19','E20','E21','E22']

In [28]:
greenE = ['E02','E03','E06']

In [29]:
redE = ['E01','E23','E24','E25','E26']

In [30]:
print(len(yellowE))

18


## Playback devices

> Red color, total = 12

['P03','P04','P05','P07','P09','P13','P15','P22','P23','P24','P25','P26']

> Yellow color, total = 5

['P01','P02','P12','P14','P19']

> Green color, total = 9

['P06','P08','P10','P11','P16','P17','P18','P20','P21']

In [31]:
redP = ['P03','P04','P05','P07','P09','P13','P15','P22','P23','P24','P25','P26']
yellowP = ['P01','P02','P12','P14','P19']
greenP = ['P06','P08','P10','P11','P16','P17','P18','P20','P21']

In [32]:
print(len(redP))
print(len(yellowP))
print(len(greenP))

12
5
9


## Recording device

> Red color, total = 13

['R01','R05','R06','R08','R09','R10','R11','R16','R21','R22','R23','R24','R25']


> Yello color, total = 02

['R03','R15']

> Green color, total = 10

['R02','R04','R07','R12','R13','R14','R17','R18','R19','R20']

In [33]:
redR = ['R01','R05','R06','R08','R09','R10','R11','R16','R21','R22','R23','R24','R25']
yellowR = ['R03','R15']
greenR = ['R02','R04','R07','R12','R13','R14','R17','R18','R19','R20']

In [34]:
print(len(redR))
print(len(yellowR))
print(len(greenR))

13
2
10


# Replay configurations in the evaluation set

The evaluation dataset has total of 57 replay configurations. Please refer to the Hector odyssey paper to see more details on this.

In [35]:
evalProt = '/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/protocol_V2/ASVspoof2017_V2_eval.trl.txt'

In [36]:
protList = []
with open(evalProt) as f:
    for line in f:
        rc = line.strip().split(' ')[4] + ' '+  line.strip().split(' ')[5] + ' ' + line.strip().split(' ')[6]
        if rc not in protList and rc != '- - -':
            protList.append(rc)
    #print(protList)
    print(len(protList))        

57


# Inferring RC-specific file list from the score files

> Note that in most of our implementation we used genuineFirstSpoof scp and label list for the evaluation data. Here, we first extracted all the genuine trials in the beginngin and put all the spoof later (like the way trials are organised in the training and the validation sets). There is an immediate need to clean and organize the code structure with proper documentation so that few months later when you read the code you do not get confused on the implementation. Make it a bit organised !

> However, we did not create the corresponding protocal file having all the meta-data information for these files. so we will have to carefully infer these meta-data for the score files.





In [37]:
# This is the scp file where we put all genuine files first then spoof

evalScp='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/filelists/eval_genFirstSpoof.scp'

In [38]:
# The original protocal file where genuine and spoof are mixed up

evalProt='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/protocol_V2/ASVspoof2017_V2_eval.trl.txt'

In [39]:
# The model prediction scores on eval data that use evalScp for spectrogram computation
# one example

model='/homes/bc305/myphd/stage2/deeplearning.experiment1/CNN3//models_After_ICASSP/InterSpeech2018_v2.0/testing_best_model_with3sec_RELU/'
evalScore=model+'/keep_0.5_0.5_relurun8/predictions_original/eval_prediction_new.txt'

> With above function we generated a new protocal file for the evaluation data that is based on the genuineFirst
 spoof next criterion we used during spectrogram computation etc
 
> **new protocal** /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/protocol_V2/ASVspoof2017_V2_eval.trl_genFirstSpoof.txt  

> The code for producing the new protocal is here: python_codes/make_genuineFirstspoof_protocal_evalset.py

In [40]:
%%bash

#cat /homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/protocol_V2/ASVspoof2017_V2_eval.trl_genFirstSpoof.txt | head

# CASE 1: Environment specific list

> **redE** means the hardest one to detect. Please refer to the baseline paper for more details

> **yellowE** the medium hard and **green** the easiest ones

In [41]:
from eer import find_eers
from python_codes import analyse_conf

In [42]:
scoreFile = evalScore
evalScores = analyse_conf.get_scores(scoreFile)

In [43]:
evProtocal='/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/protocol_V2/ASVspoof2017_V2_eval.trl_genFirstSpoof.txt'

### i)  for greenE ['E02', 'E03', 'E06']

In [44]:
# Lets do for greenE, the easiest one first
#['E02', 'E03', 'E06']

confKey = ['E02']
saveFolder = 'E02'
gc=1298

E02_scores_spf = analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)
E02_scores_gen = evalScores[0:gc]
E02_scores = np.hstack((E02_scores_gen, E02_scores_spf))

E02_labels = np.hstack((np.ones(len(E02_scores_gen)), np.zeros(len(E02_scores_spf))))

print(len(E02_scores))


scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'

# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, E02_scores,E02_labels,saveFolder)

1939


In [45]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/E02


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   38.7762



In [46]:
confKey = ['E03']
gc=1298
saveFolder = 'E03'

E03_scores_spf = analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)
E03_scores_gen = evalScores[0:gc]
E03_scores = np.hstack((E03_scores_gen, E03_scores_spf))

E03_labels = np.hstack((np.ones(len(E03_scores_gen)), np.zeros(len(E03_scores_spf))))

print(len(E03_scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'

# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, E03_scores,E03_labels,saveFolder)

1411


In [47]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/E03


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   32.5645



In [48]:
# Lets do for greenE, the easiest one first
#['E06', 'E06', 'E06']

confKey = ['E06']
gc=1298
saveFolder = 'E06'

E06_scores_spf = analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)
E06_scores_gen = evalScores[0:gc]
E06_scores = np.hstack((E06_scores_gen, E06_scores_spf))

E06_labels = np.hstack((np.ones(len(E06_scores_gen)), np.zeros(len(E06_scores_spf))))

print(len(E06_scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'

# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, E06_scores,E06_labels,saveFolder)

1583


In [49]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/E06


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

    3.3726



In [50]:
# Lets do for greenE, the easiest one first: all three in one: E02_E03_E06

confKey = ['E02','E03','E06']
gc=1298
saveFolder = 'greenEnvironment'

# Concatenate all spoof scores
E02_E03_E06_scores_spf =  analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)
scores_gen = evalScores[0:gc]

# Put genuine scores on top
scores = np.hstack((scores_gen, E02_E03_E06_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(E02_E03_E06_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

2337


In [51]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/greenEnvironment


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   32.7170



# EER stats on greenE, the easiest ones as reported in baseline

Using the scores that was produced by our end-to-end model (we reported in Interspeech analysis paper), that showed about 5% EER on DEV and about 29.4 % on the eval set. We computed the EER with respect to the green environment (that is highly noisly and should have been easily detected by the system - thats what the baseline paper argues which use the CQCC features. But our system perform worse still, the reason is fairly simply if we corerelate our INTERSPEECH findings about the classes. )

> **E02** has 641 spoof (+1298 genuine), EER = 38.77

> **E03** has 113 spoof (+1298 genuine), EER = 32.56

> **E06** has 285 spoof (+1298 genuine), EER = 3.3726

> **greenEnvironment (E02,E03,E06) ** has 1039 spoof + 1298 genuine), EER = 32.71

### ii)  for redE = ['E01','E23','E24','E25','E26']

In [52]:
# Lets do for redE, the most difficult ones as characterized in the paper

confKey = ['E01','E23','E24','E25','E26']
gc=1298
saveFolder = 'redEnvironment'

# Concatenate all spoof scores
red_scores_spf =  analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)
scores_gen = evalScores[0:gc]

# Put genuine scores on top
scores = np.hstack((scores_gen, red_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(red_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

2931


In [53]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/redEnvironment


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   19.3090



In [54]:
%%bash
# Just to double check number of files under redE


cat '/homes/bc305/myphd/datasets/ASVSpoof2017_v2.0/protocol_V2/ASVspoof2017_V2_eval.trl.txt' | grep E26 | wc -l

178


In [55]:
748+342+183+182+178+1298

2931

# EER stats on redE, the most difficult ones

> redE = ['E01','E23','E24','E25','E26']

> **redEnvironment ** has total 2931 files (out of which first 1298 are genuine), EER = 19.3. This is very interesting as our system in this configuration seems to ***give better results than the baseline (21.86).*** See Hectors paper.

### iii)  for yellowE = ['E04','E05','E07','E08','E09','E10','E11','E12','E13','E14','E15','E16','E17','E18','E19','E20','E21','E22'] 

The medium difficulty level environment !

In [56]:
# Lets do for redE, the most difficult ones as characterized in the paper

confKey = yellowE   # see details of yellowE above
gc=1298
saveFolder='yellowEnvironment'

# Concatenate all spoof scores
yellow_scores_spf =  analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)
scores_gen = evalScores[0:gc]

# Put genuine scores on top
scores = np.hstack((scores_gen, yellow_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(yellow_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

10634


In [57]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/yellowEnvironment


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   30.7771



# Observaation on environment specific results

> **greenEnvironment (E02,E03,E06) ** has 1039 spoof + 1298 genuine), EER = 32.71

> **yellowEnvironment ** has total 10634 files (out of which first 1298 are genuine), EER = 30.77. May be these contain those files that we found earlier in our analysis work that bear genuine file characteristics. Need to double check though.

> **redEnvironment ** has total 2931 files (out of which first 1298 are genuine), EER = 19.3. This is very interesting as our system in this configuration seems to ***give better results than the baseline (21.86).*** See Hectors paper.

In [58]:
2931-1298

1633

# CASE 2: Playback device specific list

> **redE** means the hardest one to detect. Please refer to the baseline paper for more details

> **yellowE** the medium hard and **green** the easiest ones

### i)  for greenP ['P06', 'P08', 'P10', 'P11', 'P16', 'P17', 'P18', 'P20', 'P21']

In [59]:
greenP

['P06', 'P08', 'P10', 'P11', 'P16', 'P17', 'P18', 'P20', 'P21']

In [60]:
# Lets do for greenP, the easiest one first

confKey = greenP                  # pass this as a list
saveFolder='greenPlayback'
gc=1298

# Concatenate all spoof scores
green_scores_spf =  analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)

scores_gen = evalScores[0:gc]   # get genuine scores from eval scores

# Put genuine scores on top
scores = np.hstack((scores_gen, green_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(green_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

5910


In [61]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/greenPlayback/


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   42.9804



### ii)  for yellowP ['P01', 'P02', 'P12', 'P14', 'P19']

In [62]:
yellowP

['P01', 'P02', 'P12', 'P14', 'P19']

In [63]:
confKey = yellowP                  # pass this as a list
saveFolder='yellowPlayback'
gc=1298

# Concatenate all spoof scores
green_scores_spf = analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)

scores_gen = evalScores[0:gc]   # get genuine scores from eval scores

# Put genuine scores on top
scores = np.hstack((scores_gen, green_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(green_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

2866


In [64]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/yellowPlayback/


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   22.4272



### iii)  for redP  ['P03', 'P04', 'P05', 'P07', 'P09', 'P13', 'P15', 'P22', 'P23', 'P24', 'P25', 'P26']

In [65]:
redP

['P03',
 'P04',
 'P05',
 'P07',
 'P09',
 'P13',
 'P15',
 'P22',
 'P23',
 'P24',
 'P25',
 'P26']

In [66]:
confKey = redP                  # pass this as a list
saveFolder='redPlayback'
gc=1298

# Concatenate all spoof scores
green_scores_spf =  analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)

scores_gen = evalScores[0:gc]   # get genuine scores from eval scores

# Put genuine scores on top
scores = np.hstack((scores_gen, green_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(green_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

7126


In [67]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/redPlayback/


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   17.4575



**Observations on Playback devices**

> **greenP** has 5910 total files (first 1298 are genuine and remaining spoof). EER = 42.98

> **yellowP** has 2866 total files (first 1298 are genuine and remaining spoof). EER = 22.42

> **redP** has 7126 total files (first 1298 are genuine and remaining spoof). EER = 17.45

In [68]:
7126-1298

5828

# CASE 3: Recording device specific list

> **redE** means the hardest one to detect. Please refer to the baseline paper for more details

> **yellowE** the medium hard and **green** the easiest ones

### i)  for greenR: ['R02', 'R04', 'R07', 'R12', 'R13', 'R14', 'R17', 'R18', 'R19', 'R20']

In [69]:
greenR

['R02', 'R04', 'R07', 'R12', 'R13', 'R14', 'R17', 'R18', 'R19', 'R20']

In [70]:
confKey = greenR                  # pass this as a list
saveFolder='greenRecording'
gc=1298

# Concatenate all spoof scores
green_scores_spf = analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)

scores_gen = evalScores[0:gc]   # get genuine scores from eval scores

# Put genuine scores on top
scores = np.hstack((scores_gen, green_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(green_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

6390


In [71]:
6390-1298

5092

In [72]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/greenRecording/


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   33.0372



### ii)  for yellowR: ['R03', 'R15']

In [73]:
yellowR

['R03', 'R15']

In [74]:
confKey = yellowR                  # pass this as a list
saveFolder='yellowRecording'
gc=1298

# Concatenate all spoof scores
green_scores_spf = analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)

scores_gen = evalScores[0:gc]   # get genuine scores from eval scores

# Put genuine scores on top
scores = np.hstack((scores_gen, green_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(green_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

2890


In [75]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/yellowRecording/


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   31.9441



### iii)  for redR: ['R01', 'R05', 'R06', 'R08', 'R09', 'R10', 'R11', 'R16', 'R21', 'R22', 'R23', 'R24', 'R25']

In [76]:
redR

['R01',
 'R05',
 'R06',
 'R08',
 'R09',
 'R10',
 'R11',
 'R16',
 'R21',
 'R22',
 'R23',
 'R24',
 'R25']

In [77]:
confKey = redR                  # pass this as a list
saveFolder='redRecording'
gc=1298

# Concatenate all spoof scores
green_scores_spf = analyse_conf.get_config_specific_scores(scoreFile, evProtocal, confKey)

scores_gen = evalScores[0:gc]   # get genuine scores from eval scores

# Put genuine scores on top
scores = np.hstack((scores_gen, green_scores_spf))

# Make the labels
labels = np.hstack((np.ones(len(scores_gen)), np.zeros(len(green_scores_spf))))

print(len(scores))
scoreSavePath = model+'/keep_0.5_0.5_relurun8/environment_wise/'


# Save the score file and label file
analyse_conf.save_scores_labels(scoreSavePath, scores, labels, saveFolder)

6622


In [78]:
%%bash

python eer.py keep_0.5_0.5_relurun8/environment_wise/redRecording/


                            < M A T L A B (R) >
                  Copyright 1984-2018 The MathWorks, Inc.
                   R2018a (9.4.0.813654) 64-bit (glnxa64)
                             February 23, 2018

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 

EER =

   24.2499



**Observations on Recording devices**

> **greenR** has 6390 total files (first 1298 are genuine and remaining spoof). EER = 33.03

> **yellowR** has 2890 total files (first 1298 are genuine and remaining spoof). EER = 31.9

> **redR** has 6622 total files (first 1298 are genuine and remaining spoof). EER = 24.24

In [79]:
6622-1298

5324